# 1. Required Libraries Imports

In [17]:
import dash
import numpy as np
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from datetime import datetime
import pandas as pd
from scipy import optimize
from scipy import integrate
%matplotlib inline
import matplotlib as mpl
import seaborn as sns
import random
sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

# 2. Updating and Storing John Hopkins Covid Data

In [18]:
def store_flat_table_JH_data():

    data_path_1 = r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/" \
                  r"master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    pd_raw = pd.read_csv(data_path_1)
    time_idx = pd_raw.columns[4:]
    df_plot = pd.DataFrame({"date": time_idx})
    Country_list = ["Italy", "India", "US", "Germany"]
    for each in Country_list:
        df_plot[each] = np.array(pd_raw[pd_raw["Country/Region"] == each].iloc[:, 4::].sum(axis=0))

    time_idx = [datetime.strptime(each, "%m/%d/%y") for each in df_plot.date]
    time_str = [each.strftime('%Y-%m-%d') for each in time_idx]
    df_plot["date"] = time_idx
    df_plot.to_csv('../data/processed/COVID_small_flat_table.csv', sep=';', index=False)
    print("Data Acquired and Stored as COVID_small_flat_table.csv")


store_flat_table_JH_data()

Data Acquired and Stored as COVID_small_flat_table.csv


# 3. SIR Modelling

In [19]:
def SIR_model_t(SIR,t,beta,gamma):
    S,I,R = SIR
    dS_dt = -beta*S*I/N0
    dI_dt = beta*S*I/N0-gamma*I
    dR_dt = gamma*I
    return dS_dt,dI_dt,dR_dt


def slice_data(data,con):
    ydata = np.array(data[con])
    t = np.arange(len(ydata))
    return ydata, t

def fit_odeint(x, beta, gamma):
    I0 = ydata[0]
    S0 =N0 - I0
    R0=0
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1]

# 4. Applying SIR Model for various countries

In [20]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';')
df_analyse=df_analyse.sort_values('date',ascending=True)
df_analyse=df_analyse[30:140:]
df_analyse= df_analyse.reset_index().drop(["index"], axis=1)
N0 =10000000
popt=[0.3,0.1]
country_list= df_analyse.columns[1:]
for each in country_list:
    ydata, t = slice_data(df_analyse,each)
    fit_odeint(t, *popt)
    popt2, pcov = optimize.curve_fit(fit_odeint, t, ydata, maxfev=1000)
    perr = np.sqrt(np.diag(pcov))
    print('standard deviation errors : ', str(perr), ' start infect:', ydata[0])
    print("Optimal parameters: beta =", popt2[0], " and gamma = ", popt2[1])
    fitted=fit_odeint(t, *popt)
    df_analyse[each+"fit"]= pd.DataFrame(fitted)

df_analyse= df_analyse.reset_index()
# df_analyse.head()


E:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning:

overflow encountered in double_scalars

E:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning:

overflow encountered in double_scalars

E:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning:

overflow encountered in double_scalars



standard deviation errors :  [0.01405656 0.01405886]  start infect: 20
Optimal parameters: beta = 0.5427358614782404  and gamma =  0.4143119357954698
standard deviation errors :  [0.00739584 0.00708573]  start infect: 3
Optimal parameters: beta = 0.5700248555164169  and gamma =  0.44961382701901825
standard deviation errors :  [0.00428435 0.00410682]  start infect: 16
Optimal parameters: beta = 0.2591928848922302  and gamma =  0.11097663393878249
standard deviation errors :  [0.01529899 0.0152979 ]  start infect: 16
Optimal parameters: beta = 0.6078025964627297  and gamma =  0.4806190436318125


# 5. Dynamic Visualization of SIR Curves

In [ ]:

color_list = []
for i in range(200):
    var = '#%02x%02x%02x'%(random.randint(0,255),random.randint(0,255),random.randint(0,255))
    color_list.append(var)

df_input_large=df_analyse

fig = go.Figure()
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to learn data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and modelling a SIR model (Susceptible-Infected and Recovered)

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for Visualizing the Simulation plots
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_analyse.columns[2:6]],
        value=['India'], # which are pre-selected
        multi=True
    ),

    dcc.Graph(figure=fig, id='main_window_slope'),


    dcc.Markdown('''

    ## What is SIR Model?
    SIR model (susceptible-infected-removed model) is a classic approach to describing the spread of contagious diseases with immunity that represents an extension of the SI model

    ![Alt Text](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/SIR_model_anim.gif/608px-SIR_model_anim.gif)

    ## Assumptions of SIR Model
    A number of assumptions are made in the SIR model:\n
    * 1) Each individual can be infected by a pathogen only once and then either become immune or die.\n
    * 2) Those infected are immediately contagious, an assumption not made in the SIR model .\n
    * 3) The respective rates are constant.\n
    * 4) Those who die from the infection become like those immunized R expected.\n


    ''')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])


def update_figure(country_list):


    s=0
    my_yaxis={'type':"log",
              'title':'New Population Infected'
          }


    traces = []
    for each in country_list:

        traces.append(dict(x=df_analyse.index,
                                y=df_analyse[each],
                                mode='markers', markers = dict(color= color_list[s]),
                                opacity=0.9,
                                name=each
                        )
                )
        traces.append(dict(x=df_analyse.index,
                        y=df_analyse[each+"fit"],
                        mode='lines',line = dict(color=color_list[s]), opacity=1.0,name=each+'_Fit'))
        s= s+1

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8010)


Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

Dash is running on http://127.0.0.1:8010/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
